# Description

This notebook computes predicted expression correlations between all genes in the MultiPLIER models.

It also has a parameter set for papermill to run on a single chromosome to run in parallel (see under `Settings` below).

This notebook does not have an output because it is not directly run. If you want to see outputs for each chromosome, check out the `gene_corrs` folder, which contains a copy of this notebook for each chromosome.

# Modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy.spatial.distance import squareform
import pandas as pd
from tqdm import tqdm

import conf
from entity import Gene

# Settings

In [3]:
# mashr
EQTL_MODEL = "MASHR"
EQTL_MODEL_FILES_PREFIX = "mashr_"

# # elastic net
# EQTL_MODEL = "ELASTIC_NET"
# EQTL_MODEL_FILES_PREFIX = "en_"

# make it read the prefix from conf.py
EQTL_MODEL_FILES_PREFIX = None

# specifies a single chromosome value
# by default, run on all chromosomes
chromosome = "all"

In [4]:
# Parameters
chromosome = 4
EQTL_MODEL = "MASHR"


In [5]:
if EQTL_MODEL_FILES_PREFIX is None:
    EQTL_MODEL_FILES_PREFIX = conf.PHENOMEXCAN["PREDICTION_MODELS_PREFIXES"][EQTL_MODEL]

In [6]:
display(f"Using eQTL model: {EQTL_MODEL} / {EQTL_MODEL_FILES_PREFIX}")

'Using eQTL model: MASHR / mashr_'

In [7]:
if chromosome == "all":
    from time import sleep

    message = """
    WARNING: you are going to compute correlations of gene predicted expression across all chromosomes without parallelism.
    It is recommended that you look at the README.md file in this subfolder (nbs/08_gsa_gls/README.md) to know how to do that.
    
    It will continue in 20 seconds.
    """
    print(message)
    sleep(20)

# Load data

## MultiPLIER Z

In [8]:
multiplier_z_genes = pd.read_pickle(
    conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"]
).index.tolist()

In [9]:
len(multiplier_z_genes)

6750

In [10]:
multiplier_z_genes[:10]

['GAS6',
 'MMP14',
 'DSP',
 'MARCKSL1',
 'SPARC',
 'CTSD',
 'EPAS1',
 'PALLD',
 'PHC2',
 'LGALS3BP']

## Get gene objects

In [11]:
multiplier_gene_obj = {
    gene_name: Gene(name=gene_name)
    for gene_name in multiplier_z_genes
    if gene_name in Gene.GENE_NAME_TO_ID_MAP
}

In [12]:
len(multiplier_gene_obj)

6454

In [13]:
multiplier_gene_obj["GAS6"].ensembl_id

'ENSG00000183087'

In [14]:
_gene_obj = list(multiplier_gene_obj.values())

genes_info = pd.DataFrame(
    {
        "name": [g.name for g in _gene_obj],
        "id": [g.ensembl_id for g in _gene_obj],
        "chr": [g.chromosome for g in _gene_obj],
    }
)

In [15]:
genes_info.shape

(6454, 3)

In [16]:
genes_info.head()

,name,id,chr
0,GAS6,ENSG00000183087,13
1,MMP14,ENSG00000157227,14
2,DSP,ENSG00000096696,6
3,MARCKSL1,ENSG00000175130,1
4,SPARC,ENSG00000113140,5


## Get tissues names

In [17]:
db_files = list(conf.PHENOMEXCAN["PREDICTION_MODELS"][EQTL_MODEL].glob("*.db"))

In [18]:
assert len(db_files) == 49

In [19]:
tissues = [
    str(f).split(EQTL_MODEL_FILES_PREFIX, maxsplit=1)[1].split(".db")[0]
    for f in db_files
]

In [20]:
tissues[:5]

['Skin_Not_Sun_Exposed_Suprapubic',
 'Cells_EBV-transformed_lymphocytes',
 'Brain_Frontal_Cortex_BA9',
 'Kidney_Cortex',
 'Brain_Substantia_nigra']

# Test

In [21]:
genes_info[genes_info["chr"] == "13"]

,name,id,chr
0,GAS6,ENSG00000183087,13
139,FRY,ENSG00000073910,13
239,CCNA1,ENSG00000133101,13
283,FLT3,ENSG00000122025,13
412,TNFSF11,ENSG00000120659,13
...,...,...,...
6269,KL,ENSG00000133116,13
6351,MTMR6,ENSG00000139505,13
6379,ABCC4,ENSG00000125257,13
6416,RFXAP,ENSG00000133111,13


In [22]:
_gene_list = [
    Gene("ENSG00000134871"),
    Gene("ENSG00000187498"),
    Gene("ENSG00000183087"),
    Gene("ENSG00000073910"),
    Gene("ENSG00000133101"),
    Gene("ENSG00000122025"),
    Gene("ENSG00000120659"),
    Gene("ENSG00000133116"),
]

tissue = "Whole_Blood"

In [23]:
%%timeit
for gene_idx1 in range(0, len(_gene_list) - 1):
    gene_obj1 = _gene_list[gene_idx1]

    for gene_idx2 in range(gene_idx1 + 1, len(_gene_list)):
        gene_obj2 = _gene_list[gene_idx2]

        gene_obj1.get_expression_correlation(
            gene_obj2,
            tissue,
        )

82.1 ms ± 5.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Compute correlation per chromosome

In [24]:
all_chrs = genes_info["chr"].dropna().unique()
assert all_chrs.shape[0] == 22

if chromosome != "all":
    chromosome = str(chromosome)
    assert chromosome in all_chrs

    # run only on the chromosome specified
    all_chrs = [chromosome]

# # For testing purposes
# all_chrs = ["13"]
# tissues = ["Whole_Blood"]
# genes_info = genes_info[genes_info["id"].isin(["ENSG00000134871", "ENSG00000187498", "ENSG00000183087", "ENSG00000073910"])]

for chr_num in all_chrs:
    print(f"Chromosome {chr_num}", flush=True)

    genes_chr = genes_info[genes_info["chr"] == chr_num]
    print(f"Genes in chromosome{genes_chr.shape}", flush=True)

    gene_chr_objs = [Gene(ensembl_id=gene_id) for gene_id in genes_chr["id"]]
    gene_chr_ids = [g.ensembl_id for g in gene_chr_objs]

    n = len(gene_chr_objs)
    n_comb = int(n * (n - 1) / 2.0)
    print(f"Number of gene combinations: {n_comb}", flush=True)

    for tissue in tissues:
        print(f"Tissue {tissue}", flush=True)

        # check if results exist
        output_dir = (
            conf.PHENOMEXCAN["LD_BLOCKS"]["BASE_DIR"]
            / f"{EQTL_MODEL_FILES_PREFIX}gene_corrs"
            / tissue
        )
        output_file = output_dir / f"gene_corrs-{tissue}-chr{chr_num}.pkl"

        if output_file.exists():
            _tmp_data = pd.read_pickle(output_file)

            if _tmp_data.shape[0] > 0:
                print("Already run, stopping.")
                continue

        gene_corrs = []

        pbar = tqdm(ncols=100, total=n_comb)
        i = 0
        for gene_idx1 in range(0, len(gene_chr_objs) - 1):
            gene_obj1 = gene_chr_objs[gene_idx1]

            for gene_idx2 in range(gene_idx1 + 1, len(gene_chr_objs)):
                gene_obj2 = gene_chr_objs[gene_idx2]

                gene_corrs.append(
                    gene_obj1.get_expression_correlation(
                        gene_obj2, tissue, model_type=EQTL_MODEL
                    )
                )

                pbar.update(1)

        pbar.close()

        # testing
        gene_corrs_flat = pd.Series(gene_corrs)
        print(f"Min/max values: {gene_corrs_flat.min()} / {gene_corrs_flat.max()}")
        assert gene_corrs_flat.min() >= -1.001
        assert gene_corrs_flat.max() <= 1.001

        # save
        # FIXME: consider saving only the condenced matrix here. See here for
        # more details: https://github.com/greenelab/phenoplier/pull/38#discussion_r634600813
        gene_corrs_data = squareform(np.array(gene_corrs, dtype=np.float32))
        np.fill_diagonal(gene_corrs_data, 1.0)

        gene_corrs_df = pd.DataFrame(
            data=gene_corrs_data,
            index=gene_chr_ids,
            columns=gene_chr_ids,
        )

        output_dir.mkdir(exist_ok=True, parents=True)
        display(output_file)

        gene_corrs_df.to_pickle(output_file)

Chromosome 4
Genes in chromosome(244, 3)
Number of gene combinations: 29646
Tissue Skin_Not_Sun_Exposed_Suprapubic


100%|████████████████████████████████████████████████████████| 29646/29646 [01:40<00:00, 294.79it/s]

Min/max values: -0.9352223891230323 / 0.9457917754147297


PosixPath('/opt/data/data/phenomexcan/ld_blocks/mashr_gene_corrs/Skin_Not_Sun_Exposed_Suprapubic/gene_corrs-Skin_Not_Sun_Exposed_Suprapubic-chr4.pkl')

Tissue Cells_EBV-transformed_lymphocytes


...

PosixPath('/opt/data/data/phenomexcan/ld_blocks/mashr_gene_corrs/Nerve_Tibial/gene_corrs-Nerve_Tibial-chr4.pkl')

Tissue Liver


100%|████████████████████████████████████████████████████████| 29646/29646 [01:19<00:00, 375.24it/s]

Min/max values: -0.9177537819028923 / 0.9081957496862576


PosixPath('/opt/data/data/phenomexcan/ld_blocks/mashr_gene_corrs/Liver/gene_corrs-Liver-chr4.pkl')

# Testing

In [25]:
# data = pd.read_pickle(
#     conf.PHENOMEXCAN["LD_BLOCKS"]["BASE_DIR"] / "gene_corrs" / "Whole_Blood" / "gene_corrs-Whole_Blood-chr13.pkl"
# )

In [26]:
# assert data.loc["ENSG00000134871", "ENSG00000187498"] > 0.97